In [1]:
import re
import uuid

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from utilities import log_progress

from py2neo import Graph

graph = Graph('https://hobby-ijiahihliahcgbkepnpggmbl.dbs.graphenedb.com:24780/db/data/',
              user='remote',
              password="b.WKJhDFo38DGT.XBFHkuTL7R6BEEgw")


In [11]:
def set_note(doctrine, note):
    note = re.sub('\\x08', '\\\\b', note)
    note = re.sub('\\t', '\\\\t', note)
    note = re.sub('\\r', '\\\\r', note)
    note = re.sub(r'\\([a-z])', r'\\\\\1', note)
    q = ("MATCH (d:Doctrine {description:'%s'}) "
         "SET d.note = '%s' return d.note as note" % (doctrine, note))
    return graph.run(q).data()[0]['note']

# Utility and setting doctrines

In [12]:
q = ("MATCH (d:Doctrine)--(i:Instance)--(:DissertationPart {chapter:4}) "
     "RETURN distinct d.description, count(distinct i) as number "
     "ORDER BY number DESC")

table = graph.run(q).to_table()

In [13]:
q = ("MATCH (d:Doctrine)--(i:Instance)--(:DissertationPart {chapter:4}) "
     "WHERE NOT exists(d.note) "
     "RETURN distinct d.description, d.label, count(distinct i) as number "
     "ORDER BY number DESC")
cursor = graph.run(q)
#table = cursor.to_table()
try:
    res = cursor.data()[0]
    print(res['d.description'])
except IndexError:
    print('There are no doctrines without a note property.')
#table

There are no doctrines without a note property.


In [23]:
note = """
  \label{item:inv:doctrine:prime-matter}
  The possible intellect stands to the intelligible species as prime matter
  stands to forms. The is based on the idea of the complete potentiality of the
  intellect before the act of knowing.
"""
note = re.sub('\\x08', '\\\\b', note)
note = re.sub('\\t', '\\\\t', note)
note = re.sub('\\r', '\\\\r', note)
note = re.sub('\\f', '\\\\f', note)
note = re.sub("\'", "\\'", note)
#note = re.sub(r'\\([a-z])', r'\\\\\1', note)

print(set_note('Possible intellect similar to prime matter', note))



  \label{item:inv:doctrine:prime-matter}
  The possible intellect stands to the intelligible species as prime matter
  stands to forms. The is based on the idea of the complete potentiality of the
  intellect before the act of knowing.



In [15]:
def rename_doctrine(old, new):
    q = ("MATCH (d:Doctrine {description:'%s'}) "
         "SET d.description = '%s' "
         "RETURN d.description "% (old, new) )
    return graph.run(q).evaluate()

rename_doctrine(
    old='The agent intellect has no knowledge', 
    new='Power of the agent intellect not intelligible'
)

# Predict the labels for the appendices and set the m in the DB

In [16]:
# First, reset all uuids if necessary
ids = graph.run("MATCH (n) return id(n)").data()
for i in log_progress(ids):
    graph.run("MATCH (n) where id(n) = %i SET n.uuid = '%s' return n" % (i['id(n)'], uuid.uuid4())).data()

In [17]:
positive_q = ("""
    MATCH (i:Instance)-[:OF|SUPPORTS|IMPLIES]-(d:Doctrine)
    WHERE (not exists(i.type) or i.type = 'Conclusion' or i.type = 'Refutation') 
    AND NOT d.description = 'There can be a science of the soul'
    RETURN distinct d.description as title, d.uuid as uuid, d.note as note, count(distinct i) as usage
""")
negative_q = ("""
    MATCH (i:Instance)-[:OF|SUPPORTS|IMPLIES]-(d:Doctrine) 
    WHERE i.type = 'Negative ratio'
    AND NOT d.description = 'There can be a science of the soul'
    RETURN distinct d.description as title, d.uuid as uuid, d.note as note, count(distinct i) as usage
""")

# Remove all existing labels before we add them again
graph.run(
    "MATCH (d:Doctrine) "
    "SET d.label = []"
)

def add_labels(query, prefix=''):
    query_sorted = sorted(graph.run(query).data(), key=lambda k: k['usage'], reverse=True) 
    for i, d in enumerate(query_sorted, 1):
        label = prefix + str(i)
        out = graph.run(
            "MATCH (d) where d.uuid = '%s' "
            "SET d.label = d.label + ['%s'] " 
            "RETURN d.label" % (d['uuid'], label)).evaluate()
        print(d['title'] + ': ' + str(out))
    return query_sorted

positive_sorted = add_labels(positive_q, prefix='P')
negative_sorted = add_labels(negative_q, prefix='N')

Primary and secondary intelligibility: ['P1']
Substantial knowledge from accidents: ['P2']
Self-knowledge through external species: ['P3']
Possible intellect completely indeterminate before actualization by an external species: ['P4']
An immaterial substance is self-reflexive: ['P5']
Science about subject, parts or properties, principles: ['P6']
The intellect is self-intelligible: ['P7']
Intellect intelligble through external species: ['P8']
Agent intellect known through its products: ['P9']
Possible intellect similar to prime matter: ['P10']
Science through self-reflection: ['P11']
Great chain of being and the intellect: ['P12']
The intellect is intelligible: ['P13']
Successive self-knowledge: ['P14']
Proper and improper science: ['P15']
Form of particular can be studied as a universal: ['P16']
The soul is difficult but not impossible to know: ['P17']
Self-knowledge different from the science of the soul: ['P18']
Intellect especially difficult to know: ['P19']
Bodily senses are not re

# Produce the appendices

In [18]:
positive_inv = '\\begin{inventory}[label=P\\arabic*, font=\\bfseries]\n\n'

for d in positive_sorted:
    positive_inv += "\invitem{%s}%s\n" % (d['title'], d['note'])
    
positive_inv += '\end{inventory}\n\n'
print(positive_inv)

\begin{inventory}[label=P\arabic*, font=\bfseries]

\invitem{Primary and secondary intelligibility}
  Something that is not intelligible in itself can however be intelligible
  through its actualization by something else. Generalization of
  \ref{item:inv:doctrine:transference-of-intelligibility}.
  \label{item:inv:primary-secondary-intelligibility}

\invitem{Substantial knowledge from accidents}
  Stronger version of the
  self-observation thesis. Through a stepwise process the intellect acquires
  knowledge about its own substance by
  \begin{inparaenum}
  \item first observing its own handling of its primary object of knowledge,
  \item informing it about the kind of activity it engages in,
  \item this reveals which powers the intellect possesses
  \item which finally yields insights into the substance that possesses these powers.
  \end{inparaenum}
  \label{item:inv:knowledge-from-accidents}

\invitem{Self-knowledge through external species}
  \begin{enumerate}
  \item The possibl

## Negative

In [19]:
negative_inv = '\\begin{inventory}[label=N\\arabic*, font=\\bfseries]\n\n'

for d in negative_sorted:
    negative_inv += "\invitem{%s}%s\n" % (d['title'], d['note'])
    
negative_inv += '\end{inventory}\n\n'
print(negative_inv)

\begin{inventory}[label=N\arabic*, font=\bfseries]

\invitem{Soul not intelligible to us}
  \label{sec:science:inv:neg:intelligible:negative}
  \label{sec:science:inv:neg:intelligible:positive}
  \begin{enumerate}
  \item The subject of a science must be intelligible.
  \item A soul is not intelligible.
  \item $\therefore$ There cannot be a science about the soul.
  \end{enumerate}

\invitem{Soul not available to sense perception}
  \label{sec:science:inv:neg:not-perceptible:positive}
  \label{sec:science:inv:neg:not-perceptible:negative}
  \begin{enumerate}
  \item Science can only be about that which can be known, and scientific
    knowledge can only be acquired through sense perception.
  \item The soul is not available to sense perception.
  \item $\therefore$ There cannot be a science about the soul.
  \end{enumerate}

\invitem{Identity of mover and moved impossible}
  \label{challenges:phys:mover-moved}
  \begin{enumerate}
  \item The same thing cannot be mover and moved at the